# Bayesian Methods in Astronomy: Hands-on Statistics

## Big Picture: What Are We After?

There are two fundamental types of statistical questions we'll want to answer:

#### 1. Model Fitting
*Given this Model, what parameters best fit my data?*

Examples:

- What are the slope and intercept of a line of best-fit?
- What are the parameters of the best quadratic fit?
- What is the frequency, amplitude, and phase of a sinusoidal fit?
- What are the orbital parameters of a planet in this radial velocity data?

#### 2. Model Selection

*Given two potential Models, which better describes my data?*

Examples:

- Is there a linear trend in this data?
- Does a linear or quadratic fit describe our data better?
- Is there a periodic signal in this timeseries?
- Does this star have a planet around it? Does this star have two planets around it?

Often one of the two models is a *null hypothesis*, or a baseline model in which the effect you're interested in is not observed.

## Frequentist vs Bayesian Approaches

Both the model fitting and model selection problems can be approached from either a *frequentist* or a *Bayesian* standpoint.
Fundamentally, the difference between these lies in the **definition of probability** that they use:

- **A frequentist probability is a measure *long-run frequency* of (real or imagined) repeated trials.** Among other things, this generally means that observed data can be described probabilistically (you can repeat the experiment and get a different realization) while model parameters are fixed, and cannot be described probabilistically (the universe remains the same no matter how many times you observe it).

- **A Bayesian probability is a *quantification of belief*.** Among other things, this generally means that observed data are treated as fixed (you know exactly what values you measured) while model parameters – including the "true" values of the data reflected by noisy measurements – are treated probabilistically (your degree of knowledge about the universe changes as you gather more data).

Perhaps surprisingly to the uninitiated, people have been vehemently fighting in favor of one approach or the other for decades.
For more a more fleshed-out discussion of these different definitions and their consequences, you can see my [series of blog posts](http://jakevdp.github.io/blog/2014/03/11/frequentism-and-bayesianism-a-practical-intro/) on the topic.

## The Bayesian Problem Setting

Thus the end-goal of a Bayesian analysis is a probabilistic statement about the universe.
Roughly we want to measure

$$
P(science)
$$

Where "science" might be encapsulated in the cosmological model, the mass of a planet around a star, or whatever else we're interested in learning about.

We don't of course measure this without reference to data, so more specifically we want to measure

$$
P(science~|~data)
$$

which should be read "the probability of the science *given* the data."

Of course, we should be explicit that this measurement is not done in a vaccum: generally before observing any data we have *some* degree of background information that informs the science, so we should actually write

$$
P(science~|~data, background\ info)
$$

This should be read "the probability of the science given the data *and* the background information".

Finally, there are often things in the scientific model that we don't particularly care about: these are known as "nuisance parameters". As an example of a nuisance parameter, if you are finding a planet in radial velocity data, the secular motion of the star is *extremely* important to model correctly, but in the end you don't really care about its best-fit value.

With that in mind, we can write:

$$
P(science,nuisance\ parameters~|~data, background\ info)
$$

Where as before the comma should be read as an "and".

This is starting to get a bit cumbersome, so let's create some symbols that will let us express this more easily:

$$
P(\theta_S, \theta_N~|~D, I)
$$

- $\theta_S$ represents the "science": the vector of parameters that we are interested in constraining
- $\theta_N$ represents the "nuisance parameters": the vector of parameters that are important in the model, but are not particularly interesting for the scientific result.
- $D$ represents the "observed data"
- $I$ represents the information or knowledge you had before observing the data, including whatever made you choose the model you're fitting.

Finally, we'll often just write $\theta = (\theta_S, \theta_N)$ as a shorthand for all the model parameters.

This quantity, $P(\theta~|~D,I)$ is called the "posterior probability" and determining this quantity is the ultimate goal of a Bayesian analysis.

Now all we need to do is compute it!

## The Nitty Gritty: Thinking about Probability

### Normalization of Probability

That is, we can write
$$
\int P(A) dA = 1
$$

### Conditional probability

We can write
$$
P(A\mid B) = \frac{P(A, B)}{P(B)}
$$
Rearranging this we get the more common expression of this:
$$
P(A, B) = P(A\mid B)P(B)
$$

Note that conditional probabilities are still probabilities, so that the normalization is the same as above:
$$
\int P(A\mid B)dA = 1
$$

### Marginalization

Given the above two properties, we can quite easily show that
$$
\int P(A, B) dA = P(B)
$$
This is known as *marginalization* – we have integrated *A* out of the joint probability, and we are left with the raw probability of *B*.

### Bayes' Rule

The definition of conditional probability is entirely symmetric, so we can write

$$
P(A, B) = P(B, A)
$$

$$
P(A\mid B)P(B) = P(B\mid A)P(A)
$$

which is more commonly rearranged in this form:

$$
P(A\mid B) = \frac{P(B\mid A)P(A)}{P(B)}
$$

This is known as *Bayes' Theorem* or *Bayes' Rule*, and is important because it gives a formula for "flipping" conditional probabilities.

### From Bayes' Rule to Bayesian Inference

If we replace these labels, we find the usual expression of Bayes' theorem as it relates to model fitting:

$$
P(\theta \mid D) = \frac{P(D\mid\theta)P(\theta)}{P(D)}
$$

Technically all the probabilities should all be conditioned on the information $I$:

$$
P(\theta \mid D,I) = \frac{P(D \mid \theta,I)P(\theta \mid I)}{P(D \mid I)}
$$

In terms of Bayesian analysis, each of these terms have a precise name and meaning:

- $P(\theta\mid D, I)$ is the *posterior*. This is the quantity we want to compute: our knowledge of the model given the data & background knowledge (including the choice of model).
- $P(D\mid\theta,I)$ is the *likelihood*. This measures the probability of seeing our data given the model. This is identical to the expression that is maximized in frequentist *maximum-likelihood* approaches.
- $P(\theta\mid I)$ is the *prior*. This encodes any knowledge we had about the answer before measuring the current data.
- $P(D\mid I)$ is the *Fully Marginalized Likelihood*, or *FML* (also called the *Evidence* among other things). In the context of model fitting, it acts as a normalization constant and in most cases can be ignored.

### Aside on the FML

In general the Fully Marginalized Likelihood (FML) is **very costly** to compute, which makes the acronym doubly appropriate in any situation where you actually need it.
To see why it's so costly, consider that the FML can be expressed as an integral using the identities we covered above:
$$
P(D\mid I) = \int P(D\mid\theta, I) P(\theta) d\theta
$$
In other words, it is the integral over the likelihood for *all possible values of theta*.
When your likelihood is a complicated function of many parameters, computing this integral can become extremely costly (a manifestation of the *curse of dimensionality*).

### Marginalizing the nuisance parameters

We now have a recipe for computing the posterior, $P(\theta\mid D,I)$. Recall that in general our model parameters $\theta$ contain some parameters of interest (what we called $\theta_S$, or the "science") and some nuisance parameters (what we called $\theta_N$).

What we're really interested in, then, is $P(\theta_S\mid D, I)$ which we can compute via a straightforward integral:

$$
P(\theta_S\mid D, I) = \int P(\theta_S, \theta_N\mid D, I)d\theta_N
$$

where the integral is over the entire space of $\theta_N$.
This quantity, the marginalized posterior, is our final result of interest.

At first glance, you might think this problem would be just as difficult as the computation of the FML above.
We'll see later, however, that a feature of the MCMC approaches typical for large Bayesian problems is that such marginalized likelihoods come essentially for free.

## Looking forward

We now have all the probabilistic machinery we need to do Bayesian inference... next we will get our fingers on our keyboards and fit some Bayesian models!